In [ ]:
class discrete_wavelet_transform:    
    def __init__(self, raw_data, frequency_sampling):
        import numpy as np
        self.zeros = np.zeros
        self.cos = np.cos
        self.sin = np.sin
        self.pi = np.pi
        self.sqrt = np.sqrt
        self.sqr = np.sqr
        self.data = raw_data
        self.lengthdata = len(raw_data)
        self.fs = frequency_sampling
        self.nfilterbank = 5
        
    def dirac(self, x):
        if x == 0:
            return 1
        else:
            return 0

    def koef_filter_hg(self):        
        g = [self.zeros(self.data), self.zeros(self.data)] # Base array untuk array isi nol
        h = [self.zeros(self.data), self.zeros(self.data)]
        
        for i in range(0, self.data):
            for n in range(-2,1):
                g[0][i] = n
                g[1][i] = -2 * (self.dirac(n) - self.dirac(n+1))
                h[0][i] = n
                h[1][i] = 1/8 * (self.dirac(n-1) + 3*self.dirac(n) + 3*self.dirac(n) + self.dirac(n+2))
        return g, h

    def respon_frekuensi_hg(self):
        Hw = self.zeros(self.fs) # Base array untuk array isi nol
        Gw = self.zeros(self.fs)
        
        g, h = self.koef_filter_hg()
        for i in range(0, self.fs):
            reH = 0
            imH = 0
            reG = 0
            imG = 0
            for k in range(-1,2):
                reG = reG + g[k+2]*self.cos(k*2*self.pi*i/self.fs) # +2 pada k agar index tidak negatif
                imG = imG - g[k+2]*self.sin(k*2*self.pi*i/self.fs)
                reH = reH + h[k+2]*self.cos(k*2*self.pi*i/self.fs)
                imH = imH - h[k+2]*self.sin(k*2*self.pi*i/self.fs)
            Hw[i] = self.sqrt(self.sqr(reH) + self.sqr(imH))
            Gw[i] = self.sqrt(self.sqr(reG) + self.sqr(imG))
        return Hw, Gw

    def Qj_w(self):
        Q = [self.zeros(self.data), 
             self.zeros(self.data), 
             self.zeros(self.data), 
             self.zeros(self.data), 
             self.zeros(self.data), 
             self.zeros(self.data), 
             self.zeros(self.data), 
             self.zeros(self.data)]
        Hw, Gw = self.respon_frekuensi_hg() # Gw dan Hw punya panjang array yang sama
        
        for i in range(round(self.fs/2)):
            if (i*128) < len(Gw): # dipakai statement menghindari IndexError saat akses value dari array Gw dan Hw
                Q[0][i] = Gw[i]
                Q[1][i] = Gw[2*i]*Hw[i]
                Q[2][i] = Gw[4*i]*Hw[2*i]*Hw[i]
                Q[3][i] = Gw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[4][i] = Gw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[5][i] = Gw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[6][i] = Gw[64*i]*Hw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[7][i] = Gw[128*i]*Hw[64*i]*Hw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
            elif (i*64) < len(Gw):
                Q[0][i] = Gw[i]
                Q[1][i] = Gw[2*i]*Hw[i]
                Q[2][i] = Gw[4*i]*Hw[2*i]*Hw[i]
                Q[3][i] = Gw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[4][i] = Gw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[5][i] = Gw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[6][i] = Gw[64*i]*Hw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[7][i] = Hw[64*i]*Hw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
            elif (i*32) < len(Gw):
                Q[0][i] = Gw[i]
                Q[1][i] = Gw[2*i]*Hw[i]
                Q[2][i] = Gw[4*i]*Hw[2*i]*Hw[i]
                Q[3][i] = Gw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[4][i] = Gw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[5][i] = Gw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[6][i] = Hw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[7][i] = Hw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
            elif (i*16) < len(Gw):
                Q[0][i] = Gw[i]
                Q[1][i] = Gw[2*i]*Hw[i]
                Q[2][i] = Gw[4*i]*Hw[2*i]*Hw[i]
                Q[3][i] = Gw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[4][i] = Gw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[5][i] = Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[6][i] = Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[7][i] = Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
            elif (i*8) < len(Gw):
                Q[0][i] = Gw[i]
                Q[1][i] = Gw[2*i]*Hw[i]
                Q[2][i] = Gw[4*i]*Hw[2*i]*Hw[i]
                Q[3][i] = Gw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[4][i] = Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[5][i] = Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[6][i] = Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
                Q[7][i] = Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i]
            elif (i*4) < len(Gw):
                Q[0][i] = Gw[i]
                Q[1][i] = Gw[2*i]*Hw[i]
                Q[2][i] = Gw[4*i]*Hw[2*i]*Hw[i]
                Q[3][i] = Hw[4*i]*Hw[2*i]*Hw[i]
                Q[4][i] = Hw[4*i]*Hw[2*i]*Hw[i]
                Q[5][i] = Hw[4*i]*Hw[2*i]*Hw[i]
                Q[6][i] = Hw[4*i]*Hw[2*i]*Hw[i]
                Q[7][i] = Hw[4*i]*Hw[2*i]*Hw[i]    
            elif (i*2) < len(Gw):
                Q[0][i] = Gw[i]
                Q[1][i] = Gw[2*i]*Hw[i]
                Q[2][i] = Hw[2*i]*Hw[i]
                Q[3][i] = Hw[2*i]*Hw[i]
                Q[4][i] = Hw[2*i]*Hw[i]
                Q[5][i] = Hw[2*i]*Hw[i]
                Q[6][i] = Hw[2*i]*Hw[i]
                Q[7][i] = Hw[2*i]*Hw[i]
            elif (i) < len(Gw):
                Q[0][i] = Gw[i]
                Q[1][i] = Hw[i]
                Q[2][i] = Hw[i]
                Q[3][i] = Hw[i]
                Q[4][i] = Hw[i]
                Q[5][i] = Hw[i]
                Q[6][i] = Hw[i]
                Q[7][i] = Hw[i] 
        return Q

    def Qj_k(self):
        Qj = [self.zeros(self.data), 
              self.zeros(self.data), 
              self.zeros(self.data), 
              self.zeros(self.data), 
              self.zeros(self.data)]
        
        j = 1
        start_increment = -(round(2**j) + round(2**(j-1)) - 2)
        end_increment = -(1 - round(2**(j-1)))
        delta_increment = abs(end_increment - start_increment) # dipakai selisih agar iterasi bisa mengakses index Array
        for k in range(0, delta_increment):
            Qj[0][k] = -2 * (self.dirac(k)-self.dirac(k+1))
            
        j = 2
        start_increment = -(round(2**j) + round(2**(j-1)) - 2)
        end_increment = -(1 - round(2**(j-1)))
        delta_increment = abs(end_increment - start_increment) # dipakai selisih agar iterasi bisa mengakses index Array
        for k in range(0, delta_increment):
            Qj[1][k] = -1/4 * (self.dirac(k-1) 
                                + 3*self.dirac(k) 
                                + 2*self.dirac(k+1) 
                                - 2*self.dirac(k+2) 
                                - 3*self.dirac(k+3) 
                                - self.dirac(k+4))
        
        j = 3
        start_increment = -(round(2**j) + round(2**(j-1)) - 2)
        end_increment = -(1 - round(2**(j-1)))
        delta_increment = abs(end_increment - start_increment) # dipakai selisih agar iterasi bisa mengakses index Array
        for k in range(0, delta_increment):
            Qj[2][k] = -1/32 * (self.dirac(k-3) 
                                + 3*self.dirac(k-2) 
                                + 6*self.dirac(k-1) 
                                + 10*self.dirac(k) 
                                + 11*self.dirac(k+1)
                                + 9*self.dirac(k+2) 
                                + 4*self.dirac(k+3) 
                                - 4*self.dirac(k+4) 
                                - 9*self.dirac(k+5) 
                                - 11*self.dirac(k+6)
                                - 10*self.dirac(k+7) 
                                - 6*self.dirac(k+8) 
                                - 3*self.dirac(k+9) 
                                - self.dirac(k+10))
        
        j = 4
        start_increment = -(round(2**j) + round(2**(j-1)) - 2)
        end_increment = -(1 - round(2**(j-1)))
        delta_increment = abs(end_increment - start_increment) # dipakai selisih agar iterasi bisa mengakses index Array
        for k in range(0, delta_increment):
            Qj[3][k] = -1/256 * (self.dirac(k-7) 
                                + 3*self.dirac(k-6) 
                                + 6*self.dirac(k-5) 
                                + 10*self.dirac(k-4) 
                                + 15*self.dirac(k-3)
                                + 21*self.dirac(k-2) 
                                + 28*self.dirac(k-1) 
                                + 36*self.dirac(k) 
                                + 41*self.dirac(k+1) 
                                + 43*self.dirac(k+2)
                                + 42*self.dirac(k+3) 
                                + 38*self.dirac(k+4) 
                                + 31*self.dirac(k+5) 
                                + 21*self.dirac(k+6) 
                                + 8*self.dirac(k+7)
                                - 8*self.dirac(k+8) 
                                - 21*self.dirac(k+9) 
                                - 31*self.dirac(k+10) 
                                - 38*self.dirac(k+11) 
                                - 42*self.dirac(k+12)
                                - 43*self.dirac(k+13) 
                                - 41*self.dirac(k+14) 
                                - 36*self.dirac(k+15) 
                                - 28*self.dirac(k+16) 
                                - 21*self.dirac(k+17)
                                - 15*self.dirac(k+18) 
                                - 10*self.dirac(k+19) 
                                - 6*self.dirac(k+20) 
                                - 3*self.dirac(k+21) 
                                - self.dirac(k+22))
            
        j = 5
        start_increment = -(round(2**j) + round(2**(j-1)) - 2)
        end_increment = -(1 - round(2**(j-1)))
        delta_increment = abs(end_increment - start_increment) # dipakai selisih agar iterasi bisa mengakses index Array
        for k in range(0, delta_increment):
            Qj[4][k] = -1/512 * (self.dirac(k-15) 
                                + 3*self.dirac(k-14) 
                                + 6*self.dirac(k-13) 
                                + 10*self.dirac(k-12) 
                                + 15*self.dirac(k-11) 
                                + 21*self.dirac(k-10)
                                + 28*self.dirac(k-9) 
                                + 36*self.dirac(k-8) 
                                + 45*self.dirac(k-7) 
                                + 55*self.dirac(k-6) 
                                + 66*self.dirac(k-5) 
                                + 78*self.dirac(k-4)
                                + 91*self.dirac(k-3) 
                                + 105*self.dirac(k-2) 
                                + 120*self.dirac(k-1) 
                                + 136*self.dirac(k) 
                                + 149*self.dirac(k+1) 
                                + 159*self.dirac(k+2)
                                + 166*self.dirac(k+3) 
                                + 170*self.dirac(k+4) 
                                + 171*self.dirac(k+5) 
                                + 169*self.dirac(k+6) 
                                + 164*self.dirac(k+7) 
                                + 156*self.dirac(k+8)
                                + 145*self.dirac(k+9) 
                                + 131*self.dirac(k+10) 
                                + 114*self.dirac(k+11) 
                                + 94*self.dirac(k+12) 
                                + 71*self.dirac(k+13) 
                                + 45*self.dirac(k+14)
                                + 16*self.dirac(k+15) 
                                - 16*self.dirac(k+16) 
                                - 45*self.dirac(k+17) 
                                - 71*self.dirac(k+18) 
                                - 94*self.dirac(k+19) 
                                - 114*self.dirac(k+20)
                                - 131*self.dirac(k+21) 
                                - 145*self.dirac(k+22) 
                                - 156*self.dirac(k+23) 
                                - 164*self.dirac(k+24) 
                                - 169*self.dirac(k+25)
                                - 171*self.dirac(k+26) 
                                - 170*self.dirac(k+27) 
                                - 166*self.dirac(k+28) 
                                - 159*self.dirac(k+29) 
                                - 149*self.dirac(k+30)
                                - 136*self.dirac(k+31) 
                                - 120*self.dirac(k+32) 
                                - 105*self.dirac(k+33) 
                                - 91*self.dirac(k+34) 
                                - 78*self.dirac(k+35)
                                - 66*self.dirac(k+36) 
                                - 55*self.dirac(k+37) 
                                - 45*self.dirac(k+38) 
                                - 36*self.dirac(k+39) 
                                - 28*self.dirac(k+40)
                                - 21*self.dirac(k+41) 
                                - 15*self.dirac(k+42) 
                                - 10*self.dirac(k+43) 
                                - 6*self.dirac(k+44) 
                                - 3*self.dirac(k+45)
                                - self.dirac(k+46))
        return Qj
    
    def time(self): # mencari waktu antara filter bank satu dan seterusnya
        t1 = round(2**(1-1))
        t2 = round(2**(2-1))
        t3 = round(2**(3-1))
        t4 = round(2**(4-1))
        t5 = round(2**(5-1))
        
        return t1, t2, t3, t4, t5

    def delay(self): # mencari selisih waktu antara filter bank satu dan yang lain
        t1, t2, t3, t4, t5 = self.time()
        
        delay1 = t5 - t1
        delay2 = t5 - t2
        delay3 = t5 - t3
        delay4 = t5 - t4
        delay5 = t5 - t5
        
        return delay1, delay2, delay3, delay4, delay5

    def mallat(self):
        w2fm = [self.zeros(self.data), 
                self.zeros(self.data), 
                self.zeros(self.data), 
                self.zeros(self.data), 
                self.zeros(self.data)] # Base array untuk array isi nol
        s2fm = w2fm
        g, h = self.koef_filter_hg()
        
        for n in range(0, self.lengthdata):
            for j in range(0, self.nfilterbank):
                temp1, temp2 = 0, 0
                for k in range(-1, 2):
                    temp1 = temp1 + g[k+1]*self.data[round(n-2**(j-1)*k)] # +1 pada k agar index tidak negatif
                    temp2 = temp2 + h[k+1]*self.data[round(n-2**(j-1)*k)] # +1 pada k agar index tidak negatif
                w2fm[j][n] = temp1
                s2fm[j][n] = temp2
        
        return w2fm, s2fm

    def filter_bank(self):        
        Qj = self.Qj_k()
        t1, t2, t3, t4, t5 = self.time() 
        
        w2fb = [self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata)] # Base array untuk array isi nol
        gradien = [self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata)]
        result = [self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata)]
        zerocross = [self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata)]
        
        w2fbmax = self.zeros(self.nfilterbank) # Base array untuk array isi nol
        w2fbmin = self.zeros(self.nfilterbank)
        
        resultQRS = self.zeros(self.lengthdata) # Base array untuk array isi nol
        resultT = self.zeros(self.lengthdata)
        eventR = self.zeros(self.lengthdata)
        eventT = self.zeros(self.lengthdata)
        eventRT = self.zeros(self.lengthdata)
        eventP = self.zeros(self.lengthdata)
        pulseevent = self.zeros(self.lengthdata)
        savevalueP2 = self.zeros(self.lengthdata)
        savevaluePa = self.zeros(self.lengthdata)
        
        flagP = 0
        pointP = 0
        
        ### FILTERBANK
        for n in range(0, self.lengthdata):
            for j in range(1, self.nfilterbank): # adanya pengurangan di perhitungan start_increment
                start_increment = -(round(2**j) + round(2**(j-1)) - 2)
                end_increment = -(1 - round(2**(j-1)))
                for k in range(start_increment, end_increment):
                    if (n-k) < self.lengthdata: # agar tidak IndexError
                        if (n+t1) < self.lengthdata: 
                            w2fb[0][n + t1] = w2fb[0][n + t1] + Qj[i][k]*self.data[n-k] # perulangan kode biar gampang dibaca
                        if (n+t2) < self.lengthdata:
                            w2fb[1][n + t2] = w2fb[1][n + t2] + Qj[1][k]*self.data[n-k]
                        if (n+t3) < self.lengthdata:
                            w2fb[2][n + t3] = w2fb[2][n + t3] + Qj[2][k]*self.data[n-k]
                        if (n+t4) < self.lengthdata:    
                            w2fb[3][n + t4] = w2fb[3][n + t4] + Qj[3][k]*self.data[n-k]
                        if (n+t5) < self.lengthdata:
                            w2fb[4][n + t5] = w2fb[4][n + t5] + Qj[4][k]*self.data[n-k]
        self.w2fb = w2fb # agar bisa diakses pada function lain dalam class
        
        for j in range(0, self.nfilterbank):
            w2fbmax[j] = max(w2fb[j])
            w2fbmin[j] = min(w2fb[j])
             
        ### MAXIMA MINIMA    
        for n in range(0, self.lengthdata):
            for j in range(0, self.nfilterbank):
                gradien[j][n] = w2fb[j][n - (j+1)] - w2fb[j][n + (j+1)] # j+1 untuk perulangan gradien satu dan seterusnya
            
            if gradien[0][n] > 0.7:
                result[0][n] = 1
            else:
                result[0][n] = 0

            if gradien[1][n] > 0.7:
                result[1][n] = 1
            else:
                result[1][n] = 0
        
            if gradien[2][n] > 0.7:
                result[2][n] = 1
            else:
                result[2][n] = 0
        
            if gradien[3][n] > 0.1:
                result[3][n] = 1
            else:
                result[3][n] = 0
        
            if gradien[4][n] > 0.7:
                result[4][n] = 1
            else:
                result[4][n] = 0
        
        ### DETEKSI PULSE QRS T
        for n in range(0, self.lengthdata):
            resultQRS[n] = result[0][n] or result[1][n] or result[2][n]
            resultT[n] = result[4][n]
        
        ### ZERO CROSSING    
        for n in range(1, self.lengthdata): # iterasi mulai dari satu agar bisa membandingkan nilai sebelumnya
            if (w2fb[0][n-1] >= 0.3) and (-0.3 >= w2fb[0][n]):
                zerocross[0][n] = 2
            else:
                zerocross[0][n] = 0
                
            if (w2fb[1][n-1] >= 0.1) and (-0.1 >= w2fb[1][n]):
                zerocross[1][n] = 2
            else:
                zerocross[1][n] = 0
                
            if (w2fb[2][n-1] >= 0.1) and (-0.1 >= w2fb[2][n]):
                zerocross[2][n] = 2
            else:
                zerocross[2][n] = 0
                
            if (w2fb[3][n-1] > 0) and (0 > w2fb[3][n]):
                zerocross[3][n] = 1
            else:
                zerocross[3][n] = 0

            if (w2fb[4][n-1] > 0) and (0 > w2fb[4][n]):
                zerocross[4][n] = 2
            else:
                zerocross[4][n] = 0
        
        ### DETEKSI P        
        for n in range(0, self.lengthdata):
            eventR[n] = zerocross[3][n] and resultQRS[n]
            eventT[n] = zerocross[3][n] and resultT[n]
            eventRT[n] = eventR[n] or eventT[n]
            eventP[n] = zerocross[3][n] ^ eventRT[n]
            pulseevent[n] = eventP[n] or result[3][n]
        
        k = 0
        for n in range(0, self.lengthdata):
            if flagP == 0:
                if pulseevent[n] == 1:
                    k = k + 1
                    if eventP[n] == 1:
                        pointP = 1
                    else:
                        pointP = 2
                else:
                    if pointP == 1:
                        flagP = 1
                    else:
                        flagP = 2
            
            if flagP == 1:
                for i in range(0, k):
                    savevalueP2[n+i-k] = 1
                flagP = 0
                k = 0
            
            if flagP == 2:
                for i in range(0, k):
                    savevalueP2[n+i-k] = 0
                flagP = 0
                k = 0
        for n in range(0, self.lengthdata):
            savevaluePa[n] = pulseevent[n] * savevalueP2[n]
        
    #kurang deteksi P QRS T

    def detection_architecture(self):
        w2fbb = [self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata)]
        w2fbbmav = [self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata)] # moving average dari depan
        w2fbbmavr = [self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata), 
                self.zeros(self.lengthdata)] # moving average dari belakang
        w2fbbmax = self.zeros(self.nfilterbank)
        window = 10
        
        for n in range(0, self.lengthdata):
            for j in range(0, self.nfilterbank):
                w2fbb[j][n] = abs(self.w2fb[j][n])
                
        for n in range(0, self.lengthdata):
            for i in range(0, window):
                if (n-i) < self.lengthdata: # agar tidak IndexError
                    w2fbbmav[0][n] = w2fbbmav[0][n] + w2fbb[0][n-i]/window
        
        for n in range(self.lengthdata, 0, -1): # berjalan dari belakang
            for i in range(0, window):
                if (n+i) < self.lengthdata: 
                    w2fbbmavr[0][n] = w2fbbmavr[0][n] + w2fbb[0][n+i]/window
                
        for j in range(0, self.nfilterbank):
            w2fbbmax[j] = max(w2fbb[j])
                    
    